In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.io import wavfile
import scipy.signal

%matplotlib tk

# fig, axs = plt.subplots(4, 1)

sampling_rate, data = wavfile.read('lab_male.wav')


# axs[0].title.set_text("lab_male.wav")
# axs[0].plot(range(len(data)), data)

# sampling_rate, data = wavfile.read('lab_female.wav')

# axs[1].title.set_text("lab_female.wav")
# axs[1].plot(range(len(data)), data)

# sampling_rate, data = wavfile.read('studio_male.wav')

# axs[2].title.set_text("studio_male.wav")
# axs[2].plot(range(len(data)), data)

# sampling_rate, data = wavfile.read('studio_female.wav')

# axs[3].title.set_text("studio_female.wav")
# axs[3].plot(range(len(data)), data)

# fig.text(0.1, 0.04, 'Timestamp', ha='center', va='center')
# fig.text(0.06, 0.15, 'Amptitude', ha='center', va='center', rotation='vertical')


In [2]:
def find_baseline(data, sampling_rate, PRODUCTS):

    '''
    
    To tell the difference of the non-voice and voice part of the signal
    we have to form a baseline based on the power spectrum.
    
    This function assume the first 2000 samples is in the non-voice part of the signal.
    It'll use the Harmonic Product Spectrum function to generate the corresponding baseline.
    
    Input: 
        data : array-like            
                        Full-length signal.
        sampling_rate : int  
                        Sampling rate of the signal.
        PRODUCTS : int 
                        Number of products to evaluate the harmonic product spectrum over.
    
    Output:
        maxv : int 
                        Base energy level that the algorithm based on to filter out the non-voice part's f0.
    
    '''
    
    WINDOW_SIZE = 2000
    window = np.ones(WINDOW_SIZE)
    i=1
    get_data_slice = data[int((0.5*i-0.5)*WINDOW_SIZE) : int((0.5*i+0.5)*WINDOW_SIZE)]

    fourier_transform = np.fft.rfft(get_data_slice*window, 2**13)

    abs_fourier_transform = np.abs(fourier_transform)
    power_spectrum = np.square(abs_fourier_transform)

    down = []
    for i in range(1, PRODUCTS+1):
        down.append(power_spectrum[::i])
    #     down1 = power_spectrum[::1]

    res = []
    for i in range(len(down[-1])):
        mul = 1
        for j in range(0, PRODUCTS):
            mul *= down[j][i]
        res.append(mul)
    
    maxv = max(res)
    
    return maxv

In [3]:
# WINDOW_SIZE = 320
# window = np.ones(WINDOW_SIZE)
# FFT_POINT = 2**13
# PRODUCTS = 10
# i=41
# get_data_slice = data[16000 : 16000+WINDOW_SIZE]
# _ = []

# fourier_transform = (np.fft.rfft(get_data_slice*window, n=FFT_POINT))
# abs_fourier_transform = np.abs(fourier_transform)
# power_spectrum = np.square(abs_fourier_transform)
# frequency = np.linspace(0, sampling_rate/2, num=len(fourier_transform))
# base = find_baseline(data, sampling_rate, PRODUCTS)

# down = []
# for i in range(1, PRODUCTS+1):
#     down.append(power_spectrum[::i])

# res = [] 
# for i in range(len(down[-1])):
#     mul = 1
#     for j in range(0, PRODUCTS):
#         mul *= down[j][i]
#     res.append(mul)

# maxv = max(res)
# indv = res.index(maxv)


# # if frequency[indv] < LOW_PASS or frequency[indv] > HIGH_PASS:
# #     _.append(0)
    
# # else:
# _.append(frequency[indv])

# fig, axs = plt.subplots(4, 1)
# axs[0].title.set_text("A window with size 320 sample of lab_male.wav using rectangular window")
# axs[0].set_xlabel("Timestamps (samples)")
# axs[0].set_ylabel("Amplitude")
# axs[0].plot(range(len(get_data_slice)), get_data_slice*window)

# axs[1].title.set_text("Power Spectrum using rectangular window")
# axs[1].set_ylabel("Power")
# axs[1].set_xlabel("Frequency")
# axs[1].plot(frequency, power_spectrum)
# print(_)

In [4]:
WINDOW_SIZE = 320
window = np.hanning(WINDOW_SIZE)
FFT_POINT = 2**17
PRODUCTS = 10
i=1
get_data_slice = data[16000 : 16000+WINDOW_SIZE]
_ = []

fourier_transform = (np.fft.rfft(get_data_slice*window, n=FFT_POINT))
abs_fourier_transform = np.abs(fourier_transform)
power_spectrum = np.square(abs_fourier_transform)
# frequency = np.linspace(0, sampling_rate/2, num=len(fourier_transform))
frequency = (np.fft.rfftfreq(len(fourier_transform)*2-1,1/sampling_rate))
base = find_baseline(data, sampling_rate, PRODUCTS)

down = []
for i in range(1, PRODUCTS+1):
    down.append(power_spectrum[::i])

res = [] 
for i in range(len(down[-1])):
    mul = 1
    for j in range(0, PRODUCTS):
        mul *= down[j][i]
    res.append(mul)

maxv = max(res)
indv = res.index(maxv)


# if frequency[indv] < LOW_PASS or frequency[indv] > HIGH_PASS:
#     _.append(0)
    
# else:
fig, axs = plt.subplots(3, 1)

_.append(frequency[indv])

axs[0].title.set_text("A window with size 740 sample of lab_male.wav non-voice")
axs[0].set_xlabel("Timestamps (samples)")
axs[0].set_ylabel("Amplitude")
axs[0].plot(range(len(get_data_slice)), get_data_slice*window)

axs[1].title.set_text("Power Spectrum apply hanning window")
axs[1].set_ylabel("Power")
axs[1].set_xlabel("Frequency")
axs[1].plot(frequency, power_spectrum)

axs[2].title.set_text("Power Spectrum after downsample by product_num of 5 and multiply")
axs[2].set_ylabel("Power")
axs[2].set_xlabel("Frequency")
axs[2].plot(range(len(res)), res)
axs[2].plot(indv, maxv, 'o')



print(frequency[indv])

116.3321202688578


In [5]:
print(indv)

953
